# Importing Libraries

In [1]:
import urllib.request as urllib2
import json
import time
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split  

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Thingspeak Configuration

In [3]:
READ_API_KEY= '95N3Z9ICXXH7RV3T'
CHANNEL_ID= '2336546'

In [4]:
# while True:
TS = urllib2.urlopen("http://api.thingspeak.com/channels/%s/feeds/last.json?api_key=%s" \
                       % (CHANNEL_ID,READ_API_KEY))

response = TS.read()
data=json.loads(response)


a = data['created_at']
b = data['field1']
c = data['field2']
d = data['field3']
e = data['field4']
f = data['field5']
g = data['field6']
print (a + "    " + b + "    " + c + "    " + d + "    " + e + "    " + f + "    " + g)
time.sleep(5)   

TS.close()

2023-11-07T19:20:48Z    30.90    43.00    16.84    2976.27    46    1023.00






# Reading the Dataset

In [5]:
ds=pd.read_csv("weatherAUS.csv")  

In [6]:
ds.head()    

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


# Cleaning the Dataset

In [7]:
ds = ds.drop(columns = ['Location','Evaporation', 'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am','WindDir3pm','WindSpeed9am', 'WindSpeed3pm', 'Cloud9am', 'Cloud3pm'])

In [8]:
ds.head()

,Date,MinTemp,MaxTemp,Rainfall,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,13.4,22.9,0.6,71.0,22.0,1007.7,1007.1,16.9,21.8,No,No
1,2008-12-02,7.4,25.1,0.0,44.0,25.0,1010.6,1007.8,17.2,24.3,No,No
2,2008-12-03,12.9,25.7,0.0,38.0,30.0,1007.6,1008.7,21.0,23.2,No,No
3,2008-12-04,9.2,28.0,0.0,45.0,16.0,1017.6,1012.8,18.1,26.5,No,No
4,2008-12-05,17.5,32.3,1.0,82.0,33.0,1010.8,1006.0,17.8,29.7,No,No


In [9]:
ds['Humidity'] = ds[['Humidity9am', 'Humidity3pm']].mean(axis=1)
ds['Pressure'] = ds[['Pressure9am', 'Pressure3pm']].mean(axis=1)
ds['Temperature'] = ds[['Temp9am', 'Temp3pm']].mean(axis=1)
ds = ds.drop(columns = ['Humidity9am', 'Humidity3pm','Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm'])
new_cols = ['Date','MinTemp', 'MaxTemp', 'Humidity', 'Pressure', 'Temperature', 'RainToday', 'RainTomorrow']
ds = ds.reindex(columns=new_cols)
ds.head()

,Date,MinTemp,MaxTemp,Humidity,Pressure,Temperature,RainToday,RainTomorrow
0,2008-12-01,13.4,22.9,46.5,1007.40,19.35,No,No
1,2008-12-02,7.4,25.1,34.5,1009.20,20.75,No,No
2,2008-12-03,12.9,25.7,34.0,1008.15,22.10,No,No
3,2008-12-04,9.2,28.0,30.5,1015.20,22.30,No,No
4,2008-12-05,17.5,32.3,57.5,1008.40,23.75,No,No


In [10]:
ds.isnull().sum()

Date                0
MinTemp          1485
MaxTemp          1261
Humidity         1887
Pressure        14804
Temperature      1129
RainToday        3261
RainTomorrow     3267
dtype: int64

In [11]:
ds = ds.dropna()

In [12]:
ds.isnull().sum()

Date            0
MinTemp         0
MaxTemp         0
Humidity        0
Pressure        0
Temperature     0
RainToday       0
RainTomorrow    0
dtype: int64


# Splitting the Dependent and Independent Variables

In [13]:
x = ds.iloc[:,1:7]
display(x.head())

,MinTemp,MaxTemp,Humidity,Pressure,Temperature,RainToday
0,13.4,22.9,46.5,1007.40,19.35,No
1,7.4,25.1,34.5,1009.20,20.75,No
2,12.9,25.7,34.0,1008.15,22.10,No
3,9.2,28.0,30.5,1015.20,22.30,No
4,17.5,32.3,57.5,1008.40,23.75,No


In [14]:
y = ds.iloc[:,7]
display(y.head())

0    No
1    No
2    No
3    No
4    No
Name: RainTomorrow, dtype: object

# One Hot Encoding

In [15]:
x_encoded = pd.get_dummies(x, columns=['RainToday'], drop_first=True)
x_encoded.rename(columns = {'RainToday_Yes':'RainToday'}, inplace = True)
x_encoded.head()

,MinTemp,MaxTemp,Humidity,Pressure,Temperature,RainToday
0,13.4,22.9,46.5,1007.40,19.35,0
1,7.4,25.1,34.5,1009.20,20.75,0
2,12.9,25.7,34.0,1008.15,22.10,0
3,9.2,28.0,30.5,1015.20,22.30,0
4,17.5,32.3,57.5,1008.40,23.75,0


In [16]:
y_encoded = pd.get_dummies(y, columns=['RainTomorrow'], drop_first=True)
y_encoded.rename(columns = {'Yes':'RainTomorrow'}, inplace = True)
y_encoded.head()

,RainTomorrow
0,0
1,0
2,0
3,0
4,0


# Splitting the Train and Test Data

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x_encoded, y_encoded, test_size = 0.20, random_state=0)  

# Model Building

In [18]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(x_train,y_train)

RandomForestClassifier()

# Predicting using Test Data

In [19]:
pred = model_rf.predict(x_test)

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred, normalize = True)

0.8317427632360511

# Predicting for the Actual Data from the Sensors

In [21]:
sensor_data = pd.read_csv("feeds.csv")

In [22]:
sensor_data.head()

,created_at,entry_id,field1,field2,field3,field4,field5,field6,latitude,longitude,elevation,status
0,2023-11-07T18:13:33+00:00,1,31.0,42,16.56,2977.66,42,1023.0,NaN,NaN,NaN,NaN
1,2023-11-07T18:15:41+00:00,2,36.2,32,16.85,2977.62,42,1023.0,NaN,NaN,NaN,NaN
2,2023-11-07T18:20:47+00:00,3,31.0,43,16.93,2977.48,42,1023.0,NaN,NaN,NaN,NaN
3,2023-11-07T18:21:13+00:00,4,31.0,43,16.93,2977.65,42,1023.0,NaN,NaN,NaN,NaN
4,2023-11-07T18:22:10+00:00,5,31.0,43,16.93,2977.54,42,1023.0,NaN,NaN,NaN,NaN


In [23]:
sensor_data = sensor_data.drop(columns = ['latitude','longitude', 'elevation', 'status'])

In [24]:
sensor_data.head()

,created_at,entry_id,field1,field2,field3,field4,field5,field6
0,2023-11-07T18:13:33+00:00,1,31.0,42,16.56,2977.66,42,1023.0
1,2023-11-07T18:15:41+00:00,2,36.2,32,16.85,2977.62,42,1023.0
2,2023-11-07T18:20:47+00:00,3,31.0,43,16.93,2977.48,42,1023.0
3,2023-11-07T18:21:13+00:00,4,31.0,43,16.93,2977.65,42,1023.0
4,2023-11-07T18:22:10+00:00,5,31.0,43,16.93,2977.54,42,1023.0


In [25]:
sensor_data.rename(columns = {'created_at':'Data and Time','entry_id':'Sr No.','field1':'Temperature', 'field2':'Humidity','field3':'Dew Point','field4':'Pressure','field5':'Altitude','field6':'Rainfall'},inplace = True)
sensor_data.head()

,Data and Time,Sr No.,Temperature,Humidity,Dew Point,Pressure,Altitude,Rainfall
0,2023-11-07T18:13:33+00:00,1,31.0,42,16.56,2977.66,42,1023.0
1,2023-11-07T18:15:41+00:00,2,36.2,32,16.85,2977.62,42,1023.0
2,2023-11-07T18:20:47+00:00,3,31.0,43,16.93,2977.48,42,1023.0
3,2023-11-07T18:21:13+00:00,4,31.0,43,16.93,2977.65,42,1023.0
4,2023-11-07T18:22:10+00:00,5,31.0,43,16.93,2977.54,42,1023.0


In [26]:
min_temp = sensor_data['Temperature'].min()
print('Min_Temp',min_temp)

max_temp = sensor_data['Temperature'].max()
print('Max_Temp:',max_temp)

humidity = sensor_data['Humidity'].mean()
print('Humidity:', humidity)

pressure = sensor_data['Pressure'].mean() 
print('Pressure:',pressure)

temp = sensor_data['Temperature'].mean()
print('Temperature:',temp)

if(sensor_data['Rainfall'].mean()<750):
    rain_today = 0
    print('Rainfall Today: No')
else:
    rain_today = 1
    print('Rainfall Today: Yes')

Min_Temp 30.8
Max_Temp: 36.2
Humidity: 42.84444444444444
Pressure: 2977.0068888888886
Temperature: 31.164444444444445
Rainfall Today: Yes


In [27]:
test_data = [[min_temp,max_temp,humidity,pressure,temp,rain_today]]
rain_predict = model_rf.predict(test_data)
if(rain_predict == 1):
    print("Yes it will rain tomorrow")
else:
    print("No it won't rain tomorrow")

No it won't rain tomorrow
